In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from torch import nn
import torch
import math
from sklearn.model_selection import train_test_split
import torch.utils.data as Data
from torch.nn import functional as F
import os
from torch import optim

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [3]:
BATCH_SIZE = 64
NUM_EPOCH = 100

In [4]:
fpath_positions = f"./train_labels/positions_train.npy"
fpath_train = f"./train"
fpath_val = f"./val"
fpath_test = f"./test"

In [5]:
def process_complex(X_complex):
    fX_real = np.real(X_complex)
    fX_imag = np.imag(X_complex)
    fX_abs = np.abs(X_complex)
    tX = np.fft.ifft(X_complex)
    tX_real = np.real(tX)
    tX_imag = np.imag(tX)
    tX_abs = np.abs(tX)
    return np.stack((fX_real, fX_imag, fX_abs, tX_real, tX_imag, tX_abs), axis=-1)

train_files = glob.glob(f"{fpath_train}/*.npy")
train_files.sort()

X = np.array([process_complex(np.load(f)) for f in train_files])
y = np.load(fpath_positions)

In [6]:
X_train, X_vali, y_train, y_vali = train_test_split(X, y, test_size=0.05, random_state=0)
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_vali = torch.Tensor(X_vali)
y_vali = torch.Tensor(y_vali)

dataset_train = Data.TensorDataset(X_train, y_train)

dataloader_train = Data.DataLoader(
    dataset = dataset_train,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2,
)

In [20]:
encoder = TransformerEncoder(100, 100, 100, 100, [64, 100], 100, 512, 10, 2, 0.1)
decoder = get_net()
model = EncoderDecoder(encoder, decoder)
model = nn.DataParallel(model)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
loss_fn = nn.MSELoss()
scheduler1 = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
scheduler2 = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 90], gamma=0.1)

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.cuda(), y.cuda()
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()        
    return loss.item()

def vali(X_vali, y_vali, model, loss_fn):
    model.eval()
    with torch.no_grad():
        X_vali, y_vali = X_vali.cuda(), y_vali.cuda()
        pred = model(X_vali)
        loss = loss_fn(pred, y_vali)
    return loss.item()

In [22]:
for t in range(NUM_EPOCH):
    print(f"Epoch {t+1}\n-------------------------------")
    loss_train = train(dataloader_train, model, loss_fn, optimizer)
    loss_vali = vali(X_vali, y_vali, model, loss_fn)
    scheduler1.step()
    scheduler2.step()
    print(loss_train, loss_vali)

Epoch 1
-------------------------------
0.06249457597732544 0.2607347071170807
Epoch 2
-------------------------------
0.013473959639668465 0.013875218108296394
Epoch 3
-------------------------------
0.015043488703668118 0.0043165963143110275
Epoch 4
-------------------------------
0.03878200054168701 0.0087971156463027
Epoch 5
-------------------------------
0.002234164159744978 0.006485289894044399
Epoch 6
-------------------------------
0.006021357607096434 0.014093856327235699
Epoch 7
-------------------------------
0.008905479684472084 0.015723034739494324
Epoch 8
-------------------------------
0.00871986709535122 0.00316994683817029
Epoch 9
-------------------------------
0.0028209583833813667 0.007677080575376749
Epoch 10
-------------------------------
0.0011253532720729709 0.001955917105078697
Epoch 11
-------------------------------
0.0017364221857860684 0.0015775476349517703
Epoch 12
-------------------------------
0.0022195284254848957 0.0018012167420238256
Epoch 13
-----

In [46]:
torch.save(model.state_dict(), 'model-512.params')

In [26]:
model.load_state_dict(torch.load('model-512.params'))

<All keys matched successfully>

In [47]:
truth = y_vali
X_vali = X_vali.cuda()
preds = model(X_vali)
truth = truth.detach().cpu().numpy()
preds = preds.detach().cpu().numpy()
# You can test the logic of the scoring mechanism on Codalab here.
#  Just be sure to keep a held-out validation set from the original training set with features and labels.
def score(truth, preds):
    # Compute the average positioning error between predictions and ground truth
    rmse = np.mean(np.sqrt(np.sum((truth - preds) ** 2, axis=1)))
    return rmse
print("Standardized RMSE: ", score(truth, preds))

Standardized RMSE:  0.012033874


In [48]:
### Predict on validation set ###

# Read CSI training files from val folder (for phase 2 of the challenge, this should be done on the test folder)
print(f"Reading validation files from {fpath_test}...")
val_files = glob.glob(f"{fpath_test}/*.npy")
val_files.sort()

# Load all files into a np array
X_val = np.array([process_complex(np.load(f)) for f in val_files])
X_val = torch.Tensor(X_val)
print("Done!")

# Predict on validation set
model.eval()
with torch.no_grad():
    X_val = X_val.cuda()
    y_val_pred = model(X_val)
print(y_val_pred.shape)

Reading validation files from ./test...
Done!
torch.Size([5000, 2])


In [49]:
# Save predictions to csv file in data_fpath folder and zip it.
#  The zipped file should be used for score submission on Codalab.
np.savetxt(f"./preds.csv", y_val_pred.cpu(), delimiter=",")
is_written = os.system(f"zip preds.zip preds.csv")

if ~is_written:
    print("Predictions saved to preds.zip")
else:
    print("Error while saving predictions to preds.zip")

updating: preds.csv (deflated 61%)
Predictions saved to preds.zip


In [7]:
class PositionWiseFFN(nn.Module):
    """基于位置的前馈网络"""
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs,
                 **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

In [8]:
class AddNorm(nn.Module):
    """残差连接后进行层规范化"""
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)

In [9]:
#@save
class DotProductAttention(nn.Module):
    """缩放点积注意力"""
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    def forward(self, queries, keys, values):
        d = queries.shape[-1]
        # 设置transpose_b=True为了交换keys的最后两个维度
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        self.attention_weights = nn.functional.softmax(scores, dim=-1)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [10]:
#@save
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，
    # num_hiddens/num_heads)
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)

    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])


#@save
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

In [11]:
#@save
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values):
        # queries，keys，values的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，
        # num_hiddens/num_heads)
        
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)

        # output的形状:(batch_size*num_heads，查询的个数，
        # num_hiddens/num_heads)
        output = self.attention(queries, keys, values)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

In [12]:
class EncoderBlock(nn.Module):
    """Transformer编码器块"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout,
            use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(
            ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X):
        Y = self.addnorm1(X, self.attention(X, X, X))
        return self.addnorm2(Y, self.ffn(Y))

In [13]:
#@save
class TransformerEncoder(nn.Module):
    """Transformer编码器"""
    def __init__(self, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, use_bias=False, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        # self.embedding = nn.Embedding(vocab_size, num_hiddens)
        # self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),
                EncoderBlock(key_size, query_size, value_size, num_hiddens,
                             norm_shape, ffn_num_input, ffn_num_hiddens,
                             num_heads, dropout, use_bias))
        # self.dense = nn.Linear(num_hiddens, 2)

    def forward(self, X, *args):
        # 因为位置编码值在-1和1之间，
        # 因此嵌入值乘以嵌入维度的平方根进行缩放，
        # 然后再与位置编码相加。
        # X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        self.attention_weights = [None] * len(self.blks)
        for i, blk in enumerate(self.blks):
            X = blk(X)
            self.attention_weights[i] = blk.attention.attention.attention_weights
        # X = self.dense(X.view([X.shape[0], -1]))
        return X

In [14]:
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [15]:
class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [16]:
def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels, use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [17]:
b1 = nn.Sequential(nn.Conv2d(6, 128, kernel_size=5, stride=2, padding=3),
                   nn.BatchNorm2d(128), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b2 = nn.Sequential(*resnet_block(128, 128, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(128, 64, 2))
# b4 = nn.Sequential(*resnet_block(128, 256, 2))
# b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [18]:
def get_net():
    net = nn.Sequential(b1, b2, b3, #b4, b5,
                        nn.AdaptiveAvgPool2d((1,1)),
                        nn.Flatten(), nn.Linear(64, 16), nn.ReLU(), nn.Linear(16, 2))
    return net

In [19]:
class EncoderDecoder(nn.Module):
    """编码器-解码器架构的基类

    Defined in :numref:`sec_encoder-decoder`"""
    def __init__(self, encoder, decoder, **kwargs):
        super(EncoderDecoder, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, X, *args):
        X1 = self.encoder(X[:,:,:,0], *args)
        X2 = self.encoder(X[:,:,:,1], *args)
        X3 = self.encoder(X[:,:,:,2], *args)
        X4 = self.encoder(X[:,:,:,3], *args)
        X5 = self.encoder(X[:,:,:,4], *args)
        X6 = self.encoder(X[:,:,:,5], *args)
        X = torch.stack((X1, X2, X3, X4, X5, X6), 1)
        output = self.decoder(X, *args)
        return output